## Reinforcement Learning Project: Highway-Env

Dieses Projekt ist entstanden in Anlehnung an die Implementierung von [Edouard Leurent](https://github.com/Farama-Foundation/HighwayEnv/blob/master/scripts/sb3_highway_dqn.ipynb).

### 1. Importing Libraries

In [1]:
# Vollständiger Import
import highway_env
import pathlib
# Alisas-Import
import gymnasium as gym
# Spezifischer Import
from stable_baselines3 import DQN
from tqdm.notebook import trange

In [2]:
%load_ext tensorboard

### 2. Defining of Constants and Paths

In [2]:
# Definition der Konstanten
BASE_MODEL_LIB = "stable_baselines3"
MODEL_TYPE = "DQN"
MODEL_VERSION = "v2"
MODEL_POLICY = "MlpPolicy"
ENVIRONMENT_NAME = "highway-v0"
RENDER_MODE = "rgb_array"

In [3]:
# Definition der Pfade
CURRENT_FILE_DIR_PATH = pathlib.Path().parent.absolute()
REPOSITORY_PATH = CURRENT_FILE_DIR_PATH.parent.absolute()
LOG_DIR_PATH = REPOSITORY_PATH.joinpath("logs", BASE_MODEL_LIB, MODEL_TYPE)
VIDEOS_DIR_PATH = REPOSITORY_PATH.joinpath("videos", BASE_MODEL_LIB, MODEL_TYPE)
MODELS_DIR_PATH = REPOSITORY_PATH.joinpath("highway_models", MODEL_TYPE, BASE_MODEL_LIB, MODEL_VERSION)

### 3. Defining of Functions

In [24]:
def train_dqn_model(
        model_policy: str,
        environment: gym.Env,
        tb_log_dir_path: str = LOG_DIR_PATH,
        model_dir_path: pathlib.Path = MODELS_DIR_PATH,
        verbose: int = 1,
        training_steps: int = 10000
) -> DQN:
    """
    Trainiert ein DQN-Modell mit der Policy MODEL_POLICY auf der Umgebung ENVIRONMENT_NAME.

    :param model_policy: Die Policy, die für das Modell verwendet werden soll.
    :param environment: Der Name der Umgebung, auf der das Modell trainiert werden soll.
    :param tb_log_dir_path: Der Pfad, in dem die Tensorboard-Logs gespeichert werden sollen.
    :param model_dir_path: Der Pfad, in dem das Modell gespeichert werden soll.
    :param verbose: Die Detailstufe der Ausgabe.
    :param training_steps: Die Anzahl der Trainingsschritte.

    :return: Das trainierte Modell.
    """
    model = DQN(model_policy,
                environment,
                policy_kwargs=dict(net_arch=[256, 256]),
                learning_rate=5e-4,
                buffer_size=15000,
                learning_starts=200,
                batch_size=32,
                gamma=0.8,
                train_freq=1,
                gradient_steps=1,
                target_update_interval=50,
                exploration_fraction=0.7,
                verbose=verbose,
                tensorboard_log=tb_log_dir_path)
    # Training des Modells
    model.learn(training_steps)
    # Speichern des Modells
    model.save(model_dir_path.joinpath(f'model_{MODEL_TYPE}_{model_policy}_{MODEL_VERSION}_{training_steps}'))
    # Rückgabe des Modells
    return model

In [8]:
def record_env(env: gym.Env, dqn_model: DQN,  video_output_folder: pathlib.Path, training: int = 10000):
    """
    Erstellt ein Video der Umgebung.
    :param env: Umgebung, die aufgezeichnet werden soll.
    :param dqn_model: Modell, das die Aktionen vorgibt.
    :param video_output_folder: Pfad, in dem das Video gespeichert werden soll.
    :param training: Anzahl der Trainingsschritte.
    :return:
    """
    # Erstellen des Video-Recorders
    wrapped_env = gym.wrappers.RecordVideo(env, video_folder=str(video_output_folder.joinpath(f'video_{MODEL_TYPE}_{MODEL_POLICY}_{MODEL_VERSION}_{training}')), episode_trigger=lambda x: True)
    env.unwrapped.set_record_video_wrapper(wrapped_env)

    for episode in trange(3, desc='Test episodes'):
        obs, info = wrapped_env.reset()
        done = truncated = False
        while not (done or truncated):
            action = dqn_model.predict(obs, deterministic=True)[0]
            obs, reward, done, truncated, info = wrapped_env.step(action)
    wrapped_env.close()

### 4. Defining of Environment

In [5]:
env_train = gym.make(ENVIRONMENT_NAME)
env_eval = gym.make(ENVIRONMENT_NAME, render_mode=RENDER_MODE)

C:\Users\Tobia\AppData\Local\Programs\Python\Python39\lib\site-packages\gymnasium\utils\passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(
C:\Users\Tobia\AppData\Local\Programs\Python\Python39\lib\site-packages\gymnasium\utils\passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(


### 5. Tensorboard Logging

In [47]:
%tensorboard --logdir "./logs/stable_baselines3/DQN"

Launching TensorBoard...

### 6. Training of Model

In [20]:
# Beispieltraining mit 100 Schritten
# model_100 = train_dqn_model("MlpPolicy", env, training_steps=100)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\logs\stable_baselines3\DQN\DQN_2
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12       |
|    ep_rew_mean      | 8.8      |
|    exploration_rate | 0.349    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2        |
|    time_elapsed     | 20       |
|    total_timesteps  | 48       |
----------------------------------


In [10]:
# Definition der einzelnen Training schritte
train_step_list_v1 = [500, 1000, 2000, 4000, 8000, 16000]
train_step_list_v2 = [32000, 48000, 64000]

In [23]:
# Definition der Modell_Speicher
model_list = []

In [25]:
# Automatisiertes Training mit den definierten Training schritten (v1)
for train_steps in train_step_list_v1:
    model = train_dqn_model(MODEL_POLICY, env_train, training_steps=train_steps)
    model_list.append(model)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\logs\stable_baselines3\DQN\DQN_9
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 7.25     |
|    ep_rew_mean      | 5.27     |
|    exploration_rate | 0.999    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2        |
|    time_elapsed     | 11       |
|    total_timesteps  | 29       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 9.5      |
|    ep_rew_mean      | 7.03     |
|    exploration_rate | 0.997    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2        |
|    time_elapsed     | 32       |
|    total_timesteps  | 76       |
----------------------------------
----------------------------------
| rollout/ 

KeyboardInterrupt: 

In [ ]:
# Automatisiertes Training mit den definierten Training schritten (v2)
for train_steps in train_step_list_v2:
    model = train_dqn_model(MODEL_POLICY, env_train, training_steps=train_steps)
    model_list.append(model)
    # Abbruch bei 64000 Schritten, da die Trainingszeit zu lange dauert und die Ergebnisse nicht mehr signifikant besser wurden (Dauer: 8h)

### 7. Evaluation of Model
#### 7.1. Loading of Model

In [6]:
# Definition des Modells entsprechend der Anzahl der Trainingsschritte
training_steps: int = 32000
# Laden des Modells
model_dqn_sb3 = DQN.load(MODELS_DIR_PATH.joinpath(f'model_{MODEL_TYPE}_{MODEL_POLICY}_{MODEL_VERSION}_{training_steps}'))

#### 7.2. Rendering of Model

In [ ]:
# Rendering des Modells
obs, info = env_eval.reset()
done = truncated = False
while not (done or truncated):
    action = model_dqn_sb3.predict(obs, deterministic=True)[0]
    obs, reward, done, truncated, info = env_eval.step(action)
    env_eval.render()

#### 7.3. Video of Model
##### Manuelle Erstellung eines Videos des Modells

In [9]:
# Erstellen eines Videos des Modells
record_env(env_eval, model_dqn_sb3, VIDEOS_DIR_PATH, training=training_steps)

C:\Users\Tobia\AppData\Local\Programs\Python\Python39\lib\site-packages\gymnasium\wrappers\record_video.py:87: UserWarning: WARN: Overwriting existing videos at D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Moviepy - Building video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000\rl-video-episode-0.mp4.
Moviepy - Writing video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000\rl-video-episode-0.mp4




t:  98%|█████████▊| 441/451 [00:01<00:00, 365.44it/s, now=None]
                                                               

Moviepy - Done !
Moviepy - video ready D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000\rl-video-episode-0.mp4
Moviepy - Building video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000\rl-video-episode-1.mp4.
Moviepy - Writing video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000\rl-video-episode-1.mp4




t:  94%|█████████▍| 397/421 [00:01<00:00, 330.23it/s, now=None]
                                                               

Moviepy - Done !
Moviepy - video ready D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000\rl-video-episode-1.mp4
Moviepy - Building video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000\rl-video-episode-2.mp4.
Moviepy - Writing video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000\rl-video-episode-2.mp4




t: 100%|█████████▉| 511/512 [00:01<00:00, 356.45it/s, now=None]
                                                               

Moviepy - Done !
Moviepy - video ready D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_32000\rl-video-episode-2.mp4


##### Automatisierte Erstellung von Videos der Modelle

In [11]:
# Definition der Modell_Speicher
loaded_model_list = {}

In [12]:
# Automatisiertes Laden der Modelle
for training_steps in train_step_list_v1:
    try:
        # Laden des Modells
        model_dqn_sb3 = DQN.load(MODELS_DIR_PATH.joinpath(f'model_{MODEL_TYPE}_{MODEL_POLICY}_{MODEL_VERSION}_{training_steps}'))
        loaded_model_list[training_steps] = model_dqn_sb3
    except:
        print(f"Das Modell mit {training_steps} Trainingsschritten konnte nicht geladen werden.")
loaded_model_list

{500: <stable_baselines3.dqn.dqn.DQN at 0x21c4a6faf70>,
 1000: <stable_baselines3.dqn.dqn.DQN at 0x21c4070e7f0>,
 2000: <stable_baselines3.dqn.dqn.DQN at 0x21c40985130>,
 4000: <stable_baselines3.dqn.dqn.DQN at 0x21c40985fa0>,
 8000: <stable_baselines3.dqn.dqn.DQN at 0x21c4a742580>,
 16000: <stable_baselines3.dqn.dqn.DQN at 0x21c4a758250>}

In [13]:
# Automatisiertes Erstellen der Videos pro Modell
for key, value in loaded_model_list.items():
    record_env(env_eval, value, VIDEOS_DIR_PATH, training=key)

C:\Users\Tobia\AppData\Local\Programs\Python\Python39\lib\site-packages\gymnasium\wrappers\record_video.py:87: UserWarning: WARN: Overwriting existing videos at D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_500 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Test episodes:   0%|          | 0/3 [00:00<?, ?it/s]

Moviepy - Building video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_500\rl-video-episode-0.mp4.
Moviepy - Writing video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_500\rl-video-episode-0.mp4




t:  83%|████████▎ | 163/196 [00:00<00:00, 351.65it/s, now=None]
                                                               

Moviepy - Done !
Moviepy - video ready D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_500\rl-video-episode-0.mp4
Moviepy - Building video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_500\rl-video-episode-1.mp4.
Moviepy - Writing video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_500\rl-video-episode-1.mp4



...

Moviepy - Done !
Moviepy - video ready D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_16000\rl-video-episode-0.mp4
Moviepy - Building video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_16000\rl-video-episode-1.mp4.
Moviepy - Writing video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_16000\rl-video-episode-1.mp4




t:  89%|████████▊ | 187/211 [00:00<00:00, 503.41it/s, now=None]
                                                               

Moviepy - Done !
Moviepy - video ready D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_16000\rl-video-episode-1.mp4
Moviepy - Building video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_16000\rl-video-episode-2.mp4.
Moviepy - Writing video D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_16000\rl-video-episode-2.mp4




t:  91%|█████████ | 549/602 [00:01<00:00, 513.75it/s, now=None]
                                                               

Moviepy - Done !
Moviepy - video ready D:\DHBW\JetBrains\Hand-on-Reinforced-Learning\videos\stable_baselines3\DQN\video_DQN_MlpPolicy_v2_16000\rl-video-episode-2.mp4
